<a href="https://colab.research.google.com/github/janithcyapa/energy-plus-utility/blob/main/energyplus-utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EnergyPlus Utility

## Installation From Github Repo and Setup Environment

In [1]:
!pip uninstall -y energy-plus-utility

In [2]:
!pip install -q "energy-plus-utility @ git+https://github.com/janithcyapa/energy-plus-utility.git@main"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


`prepare_colab_eplus` - Download and Install EnergyPlus 25-1 in `/root/EnergyPlus-22-1-0`

In [3]:
import importlib.metadata
ver = importlib.metadata.version("energy-plus-utility")
print(f"\n✅ Installed 'energy-plus-utility' version: {ver}")



✅ Installed 'energy-plus-utility' version: 0.2.1+9


In [4]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()

# Usage

In [5]:
from eplus.core import EPlusUtil
import types

### Setup Basics

In [6]:
# In your main script or core.py
# verbose, 0 - no logging, 1- verbose logging, 2 - debug logging
OUT_DIR = "/simulation/eplus_out"
sim = EPlusUtil(verbose=2, out_dir=OUT_DIR)

# Reset state before setting model
sim.reset_state()
# Delete previous output directory
sim.delete_out_dir()
# Clear previous outputs
sim.clear_eplus_outputs(patterns="eplusout.*")

Initialized StateMixin
Initialized EnergyPlus State.
Initialized IDFMixin
Initialized LoggingMixin
Initialized SimulationMixin
Initialized UtilsMixin
Initialized HandlersMixin
EnergyPlus state has been reset.
Output directory does not exist, nothing to delete: /simulation/eplus_out


### Set Model using local files

In [ ]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus-25-1-0"

# Define the Simulation Model
IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
# Select Weather Data
EPW = f"{EPLUS_DIR}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"
# Set the Model for Simulation
sim.set_model(IDF, EPW)

### Set Model From URL

In [7]:
url_idf = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/MultizoneOffice.idf"
url_epw = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/LKA_Colombo-Katunayake.434500_SWERA.epw"

# Set the Model for Simulation
sim.set_model_from_url(url_idf, url_epw)

EnergyPlus state has been reset.
Model set: IDF='/simulation/eplus_out/MultizoneOffice.idf', EPW='/simulation/eplus_out/LKA_Colombo-Katunayake.434500_SWERA.epw', OUT_DIR='/simulation/eplus_out'


### Run Basic Simualtion

In [ ]:
sim.enable_runtime_logging()
# sim.disable_runtime_logging()

In [ ]:
sim.run_dry_run(include_ems_edd=False,reset=True,design_day=False)

In [ ]:
sim.run_design_day()

In [8]:
sim.run_annual()

EnergyPlus state has been reset.


0

### Usage of Handlers

In [ ]:
# --- STEP A: DEFINE THE CUSTOM FUNCTION ---
# Notice we define it with 'self' as the first argument, just like a class method.
def dr_supervisor_logic(self, state):
    """
    A simple logic that checks the time and prints a message.
    """
    # Just printing for this demo, but you would put control logic here
    time = self.exchange.current_time(state)
    print(f"[SUPERVISOR] Checking status at time: {time:.2f}")

# --- STEP B: INJECT IT  ---
# This binds the function ONLY to this 'sim' object.
# It creates a true method where 'self' is passed automatically.
sim.my_supervisor = types.MethodType(dr_supervisor_logic, sim)


In [ ]:
print("--- 1. REGISTERING ---")
# We register the NAME of the attribute we just attached ("my_supervisor")
registered = sim.register_handlers(
    "begin",               # Hook: Begin Timestep
    ["my_supervisor"]      # Method Name
)
print(f"Registered methods: {registered}")

print("\n--- 2. LISTING ---")
# Check what is currently scheduled for the 'begin' hook
current_list = sim.list_handlers("begin")
print(f"Handlers on 'begin' hook: {current_list}")

print("\n--- 3. DISABLING (Pausing) ---")
# Scenario: It's Winter, we don't need Demand Response.
# We disable the hook so the logic stops running, but we don't delete it.
sim.disable_hook("begin")
print("Hook 'begin' is now DISABLED. (Simulation runs, but supervisor sleeps)")

print("\n--- 4. ENABLING (Resuming) ---")
# Scenario: Summer is back. Turn the logic back on.
sim.enable_hook("begin")
print("Hook 'begin' is now ENABLED. (Supervisor is active again)")

print("\n--- 5. UNREGISTERING (Deleting) ---")
# Scenario: We want to remove this logic entirely to replace it or clean up.
remaining = sim.unregister_handlers(
    "begin",
    ["my_supervisor"] # Name to remove
)
print(f"Unregistered 'my_supervisor'. Remaining handlers: {remaining}")

# --- RUN ---
# sim.run_annual()

In [ ]:
registered = sim.register_handlers(
    "begin",               # Hook: Begin Timestep
    ["my_supervisor"]      # Method Name
)

current_list = sim.list_handlers("begin")
print(f"Handlers on 'begin' hook: {current_list}")

# --- RUN ---
sim.run_annual()

## View IDF/API Data

In [ ]:
sim.api_catalog_df()

In [ ]:
sim.list_available_variables()

In [ ]:
sim.list_available_meters()

In [ ]:
sim.list_available_actuators()

In [9]:
objects_types = sim.get_idf_object_types()

print(f"{'OBJECT TYPE':<40} | {'COUNT'}")
print("-" * 50)
for obj_type, count in objects_types:
    print(f"{obj_type:<40} | {count}")

OBJECT TYPE                              | COUNT
--------------------------------------------------
No                                       | 152
Schedule:Day:Interval                    | 113
Autosize                                 | 71
Fractional                               | 64
Temperature                              | 49
Pipe:Adiabatic                           | 48
BuildingSurface:Detailed                 | 43
Branch                                   | 42
Schedule:Week:Daily                      | 38
Schedule:Year                            | 38
OnOff                                    | 37
NoSun                                    | 31
NoWind                                   | 31
Surface                                  | 26
FenestrationSurface:Detailed             | 23
Construction                             | 23
Wall                                     | 20
Window                                   | 20
Attic                                    | 19
General                 

In [17]:
object = sim.extract_idf_objects("Zone")
print(object)

['Attic ZN', 'Core_ZN ZN', 'Perimeter_ZN_1 ZN', 'Perimeter_ZN_2 ZN', 'Perimeter_ZN_3 ZN', 'Perimeter_ZN_4 ZN', '']


### Test

In [ ]:
args = [
        '-w', EPW,
        '-d', OUT_DIR,
        IDF
    ]

print("Starting simulation...")
exit_code = sim.runtime.run_energyplus(sim.state, args)

if exit_code == 0:
    print("Simulation success!")
else:
    print("Simulation failed!")

In [ ]:
print(dir(sim))
print(hasattr(sim, 'run_design_day'))